# Ingest and track data from notebook runs

Here we explain how data ingestion is tracked with notebook runs.

```{important}

`ln.nb.header()` tracks notebooks and takes care of the follwing:
1. Add a `Notebook` record `notebook` to the database
2. Add a `Run` record `run`, linked against `notebook` to the database
3. Expose `run` as `ln.nb.run`
4. Ensure that `ln.add(dobject)` sets `dobject.source = run`
5. Enable to track run inputs via `ln.DObject.load(is_run_input=True)`

```

In [ ]:
import lamindb as ln
import lamindb.schema as lns

ln.nb.header()

Query the current notebook run from DB:

In [ ]:
run = ln.select(lns.Run, id=ln.nb.run.id).one()

run

Query the current notebook entry from DB:

In [ ]:
nb = ln.select(lns.Notebook).join(lns.Run, id=ln.nb.run.id).one()

nb

## Create a DObject from notebook

The `source` parameter is automatically inferred as the current notebook run.

Alternatively, you can pass a Run record to the `source` parameter if you are ingesting from a pipeline run, see our [Pipeline guide](https://lamin.ai/docs/db/guide/pipeline).

```{note}

When creating a new DObject from [`AnnData`](https://anndata.readthedocs.io/), it defaults to save as a `h5ad` file format.

You may select `adata_format="zarr"` to ingest and store `AnnData` as [`zarr`](https://zarr.readthedocs.io/), which allows you to write objects directly from memory and read them directly into memory without saving them on disk.

```

In [ ]:
dobject = ln.DObject(
    data=ln.dev.datasets.anndata_pbmc68k_reduced(), name="pbmc68k_reduced"
)

In [ ]:
dobject.source

## Add DObject to DB

Now let's add the dobject to the database:

In [ ]:
ln.add(dobject);

## Query and load ingested data from DB

In [ ]:
dobject = ln.select(ln.DObject, name="pbmc68k_reduced").one()
adata = dobject.load()

adata

## Query data by the notebook source

Let's track where the data was ingested:

In [ ]:
ln.select(lns.Notebook).join(lns.Run).join(ln.DObject, name="pbmc68k_reduced").one()

Alternatively, you can query for the run that contains a notebook attribute:

```{admonition} What is ln.Session()?
:class: important

Why do we need session here? Find out in our [Session guide](https://lamin.ai/docs/db/faq/session).

```

In [ ]:
ss = ln.Session()

In [ ]:
source_run = ss.select(lns.Run).join(ln.DObject, name="pbmc68k_reduced").one()

source_run.notebook

In [ ]:
ss.close()